# CAO Points Analysis

## Author : Michelle O'Connor

***

In [1]:
# To access a URL from python
# import urllib

# Convenient HTTP requests
import requests as rq

# Regular expressions
import re

# Should always call this out on the requirements txt file as it doesn't come as standard package with python
# Only if you have anaconda does it come with the package

# Dates and times
import datetime as dt

# Data frames
import pandas as pd

# for downloading 
import urllib.request as urlrq

In [2]:
# Get the current date and time
now = dt.datetime.now()

# Format as a string
nowstr = now.strftime('%Y%m%d_%H%M%S')

<br>

## 2021 Points 
https://www.cao.ie/index.php?page=points&p=2021&bb=points
***

In [3]:
# Fetch the CAO points URL.
resp = rq.get('http://www2.cao.ie/points/l8.php')

#### Save Original data set

***

In [4]:
# Create a file path for the original data
pathhtml = 'data/cao2021_' + nowstr + '.html'

<br>

**Error on server**

Technically, the server says we should decode as per:
```
Content-Type: text/html; charset=iso-8859-1
```
However, one line uses \x96 which isn't defined in iso-8859-1.
<br>Therefore we use the similar decoding standard cp1252, which is very similar but includes #x96.

In [5]:
# The server uses the wrong encoding, fix it
original_encoding = resp.encoding
# Change to cp1252
resp.encoding = 'cp1252'

In [6]:
Student_dict = {
    'Name': ['Jack', 'Rack', 'Max'],
    'Marks':['100','100', '100'],
    'Fee':['100','200','300'],
    'Subject': ['Math', 'Math', 'Music']
}
  

dfobj = pd.DataFrame(Student_dict)

print(f'\n {dfobj.dtypes}')


 Name       object
Marks      object
Fee        object
Subject    object
dtype: object


In [7]:
 
dict_columns_type = {'Marks': float,
                'Fee': float
               }
   
dfobj = dfobj.astype(dict_columns_type)
print('dataframe str to float:\n',dfobj)
 
print(f'\n {dfobj.dtypes}')


dataframe str to float:
    Name  Marks    Fee Subject
0  Jack  100.0  100.0    Math
1  Rack  100.0  200.0    Math
2   Max  100.0  300.0   Music

 Name        object
Marks      float64
Fee        float64
Subject     object
dtype: object


In [8]:
Student_dict = {
    'Name': ['Jack', 'Rack', 'Max'],
    'Marks':[100,100,100],
    'Subject': ['Math', 'Math', 'Music']
}

In [9]:
dfobj = pd.DataFrame(Student_dict)

In [10]:
dfobj.dtypes

Name       object
Marks       int64
Subject    object
dtype: object

In [11]:
dfobj = dfobj.infer_objects()
print(dfobj.dtypes)

Name       object
Marks       int64
Subject    object
dtype: object


In [12]:
Student_dict = {
    'Name': ['Jack', 'Rack', 'Max'],
    'Marks':[100,100,100],
    'Subject': ['Math', 'Math', 'Music']
}
  
 
 
dfobj = pd.DataFrame(Student_dict,dtype ='object')
dfobj.dtypes

Name       object
Marks      object
Subject    object
dtype: object

In [13]:
dfobj = dfobj.infer_objects()
print(dfobj.dtypes)

Name       object
Marks       int64
Subject    object
dtype: object


In [14]:
raw_data = {'bond_name': ['govt_bond_1', 'govt_bond_2', 'govt_bond_3', 'pvt_bond_1', 'pvt_bond_2', 'pvt_bond_3', 'pvt_bond_4'],
                'risk_score': [1.6, 0.9, 2.3, 3.0, 2.7, 1.8, 4.1]}
df = pd.DataFrame(raw_data, columns = ['bond_name', 'risk_score'])
    
print(df)

     bond_name  risk_score
0  govt_bond_1         1.6
1  govt_bond_2         0.9
2  govt_bond_3         2.3
3   pvt_bond_1         3.0
4   pvt_bond_2         2.7
5   pvt_bond_3         1.8
6   pvt_bond_4         4.1


In [17]:
rating = []
for row in df['risk_score']:
    if row < 1.0 :    rating.append('AA')
    elif row < 2.0:   rating.append('A')
    elif row < 3.0:  rating.append('BB')
    elif row < 4.0:  rating.append('B')
    elif row < 5.0:  rating.append('C')
    else:           rating.append('Not_Rated')

In [18]:
df['rating'] = rating
print(df) 

     bond_name  risk_score rating
0  govt_bond_1         1.6      A
1  govt_bond_2         0.9     AA
2  govt_bond_3         2.3     BB
3   pvt_bond_1         3.0      B
4   pvt_bond_2         2.7     BB
5   pvt_bond_3         1.8      A
6   pvt_bond_4         4.1      C
